In [ ]:
!pip install "crewai[tools]"

In [ ]:
# LLM → connects to large language models (OpenAI, Gemini, etc).
# Agent → defines an “AI persona” (with role, goal, backstory).
# Task → defines what each agent needs to do.
# Crew → groups agents + tasks into a workflow.
# SerperDevTool → a tool for web search.

import os
from crewai import LLM, Agent, Task, Crew
from crewai_tools import SerperDevTool

In [ ]:
os.environ["SERPER_API_KEY"] = "your_api" ## For searching the web
os.environ["GEMINI_API_KEY"]="your_api"
os.environ["OPENAI_API_KEY"] = "your_api"
# os.environ["OPENAI_API_KEY"] = os.environ["GEMINI_API_KEY"]
os.environ["OPENAI_API_BASE"] = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature=0.1
)


In [ ]:
llm.call("what is the capital of France?")

**Agents**

In [ ]:
# This agent searches the web and collects facts.
research_assistant = Agent(
    role = "Research Assistant Agent",
    goal = "Take the research topic, conduct thorough research, and gather, organize, and summarize relevant information: {topic}",
    backstory = "A meticulous and detail-oriented researcher with strong skills in information gathering, fact-checking, and organizing sources to provide a solid foundation for writing.",
    verbose = True,
    tools = [SerperDevTool()],
    llm = llm
)

In [ ]:
# This agent writes the summary from the facts.
research_writer = Agent(
    role = "Research Writer Agent",
    goal = "Take the organized research and produce a clear, comprehensive, and well-structured research paper or report: {topic}",
    backstory = "An experienced academic writer skilled in synthesizing information into professional, coherent, and well-documented research papers and reports with clarity and precision.",
    verbose = True,
    llm = llm
)

**Tasks**

In [ ]:
task1 = Task(
    description = "Research and compile 3–5 recent and noteworthy facts about {topic} as of 2025.",
    expected_output = "A bulleted list containing 3–5 facts.",
    agent = research_assistant
)

In [ ]:
task2 = Task(
    description = "Using the collected facts, craft a concise 100-word blog-style summary about {topic}.",
    expected_output = "A well-written blog summary (around 100 words).",
    agent = research_writer,
    context = [task1],
)

**Crew**

In [ ]:
crew = Crew(
    agents = [research_assistant, research_writer],
    tasks = [task1, task2],
    verbose = True
)

In [ ]:
crew.kickoff(inputs = {"topic": "The risks of AI"})